In [4]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import LineString
import rasterio
from rasterio.plot import show
import numpy as np
from tqdm import tqdm

# === CONFIGURATION ===
TIFF_PATH = "Data/GEOTIFF_Sample.tif"
CSV_PATH = "Data/GPS_Sample.csv"
GPKG_PATH = "Data/Blocks_Sample.gpkg"
BLOCK_ID_COL = "Block_ID"

OUTPUT_ROOT = "outputs"
FRAME_DIR = os.path.join(OUTPUT_ROOT, "frames")
CSV_ANOMALY = os.path.join(OUTPUT_ROOT, "block_anomalies.csv")
CSV_SUMMARY = os.path.join(OUTPUT_ROOT, "block_total_time.csv")
VIDEO_NAME = os.path.join(OUTPUT_ROOT, "gps_animation.mp4")

WKT_COLUMN = "Location_ID"
TIME_COLUMN = "Time"
IDLE_THRESHOLD = 30  # minutes
MODERATE_THRESHOLD = 5  # minutes
FPS = 2

os.makedirs(FRAME_DIR, exist_ok=True)

# === LOAD AND PREPARE DATA ===
df = pd.read_csv(CSV_PATH)
df[TIME_COLUMN] = pd.to_datetime(df[TIME_COLUMN])
df = df.sort_values(TIME_COLUMN).reset_index(drop=True)
df["geometry"] = df[WKT_COLUMN].apply(wkt.loads)
gdf_points = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326").to_crs("EPSG:26918")

block_gdf = gpd.read_file(GPKG_PATH).to_crs("EPSG:26918")

# === CREATE LINE SEGMENTS BETWEEN POINTS ===
segments = []
anomalies = []
for i in range(1, len(gdf_points)):
    prev, curr = gdf_points.iloc[i - 1], gdf_points.iloc[i]
    duration = (curr[TIME_COLUMN] - prev[TIME_COLUMN]).total_seconds() / 60
    if duration > 0:
        linestring = LineString([prev.geometry, curr.geometry])
        if duration <= MODERATE_THRESHOLD:
            segments.append({
                "Start_Time": prev[TIME_COLUMN],
                "End_Time": curr[TIME_COLUMN],
                "Duration_Min": duration,
                "geometry": linestring
            })
        else:
            start_blocks = block_gdf[block_gdf.contains(prev.geometry)][BLOCK_ID_COL].tolist()
            end_blocks = block_gdf[block_gdf.contains(curr.geometry)][BLOCK_ID_COL].tolist()
            anomalies.append({
                "Start_Time": prev[TIME_COLUMN],
                "End_Time": curr[TIME_COLUMN],
                "Duration_Min": duration,
                "Block_Start": start_blocks[0] if start_blocks else "Non_Block",
                "Block_End": end_blocks[0] if end_blocks else "Non_Block"
            })

# Save anomalies
anomalies_df = pd.DataFrame(anomalies)
anomalies_df.to_csv(CSV_ANOMALY, index=False)

# Process only valid segments
gdf_segments = gpd.GeoDataFrame(segments, crs="EPSG:26918")

# === INTERSECT WITH BLOCKS ===
overlaps = gpd.overlay(gdf_segments, block_gdf, how="intersection", keep_geom_type=False)
overlaps["Segment_Length"] = overlaps.length
overlaps["Total_Length"] = overlaps.apply(
    lambda row: gdf_segments[
        (gdf_segments["Start_Time"] == row["Start_Time"]) &
        (gdf_segments["End_Time"] == row["End_Time"])
    ].geometry.length.values[0], axis=1
)
overlaps["Fraction"] = overlaps["Segment_Length"] / overlaps["Total_Length"]
overlaps["Time_In_Block"] = overlaps["Fraction"] * overlaps["Duration_Min"]

# === TIME SUMMARY ===
def summarize(df, label):
    return df.groupby(BLOCK_ID_COL)["Time_In_Block"].sum().rename(label)

summary = pd.concat([
    summarize(overlaps, "Valid_Time_Min")
], axis=1).fillna(0)
summary["Total_Including_Anomalies"] = summary.sum(axis=1)
summary = summary.round(2)
summary.reset_index().to_csv(CSV_SUMMARY, index=False)

# === LOAD RASTER ===
raster = rasterio.open(TIFF_PATH)

# === BUILD ANIMATION FRAMES ===
idle_periods = []
moderate_periods = []

for i in tqdm(range(len(gdf_points)), desc='Generating frames'):
    fig, ax = plt.subplots(figsize=(10, 10))
    show(raster, ax=ax)
    gdf_points.iloc[:i+1].plot(ax=ax, color='skyblue', markersize=2, alpha=0.5)

    point = gdf_points.iloc[i]
    if i > 0:
        interval = (point[TIME_COLUMN] - gdf_points.iloc[i-1][TIME_COLUMN]).total_seconds() / 60
    else:
        interval = None

    color = 'gray'
    if interval:
        if interval > IDLE_THRESHOLD:
            color = 'red'
            linestyle = '--'
            idle_periods.append(f"{gdf_points.iloc[i-1][TIME_COLUMN].strftime('%H:%M')} - {point[TIME_COLUMN].strftime('%H:%M')}")
        elif interval > MODERATE_THRESHOLD:
            color = 'yellow'
            linestyle = '--'
            moderate_periods.append(f"{gdf_points.iloc[i-1][TIME_COLUMN].strftime('%H:%M')} - {point[TIME_COLUMN].strftime('%H:%M')}")
        else:
            color = 'green'
            linestyle = '-'
    else:
        linestyle = '-'

    gpd.GeoSeries([point.geometry]).plot(ax=ax, color=color, markersize=50)

    point_block = block_gdf[block_gdf.contains(point.geometry)]
    block_id = point_block[BLOCK_ID_COL].iloc[0] if not point_block.empty else "Non_Block"

    ax.text(0.02, 0.94,
            f"Time: {point[TIME_COLUMN].strftime('%Y-%m-%d %H:%M:%S')}\nBlock: {block_id}\n∆t: {interval:.1f} min"
            if interval else
            f"First Point\nTime: {point[TIME_COLUMN].strftime('%Y-%m-%d %H:%M:%S')}\nBlock: {block_id}",
            transform=ax.transAxes, fontsize=13, fontweight='bold',
            bbox=dict(facecolor='white', alpha=0.8, edgecolor='black'))

    ax.text(0.5, 1.02, "Machine", transform=ax.transAxes,
            fontsize=10, color='white', ha='center', va='bottom',
            bbox=dict(facecolor='black', alpha=0.5, boxstyle='round,pad=0.3'))

    ax.scatter([], [], c='green', label='∆t ≤ 5 min', s=50)
    ax.scatter([], [], c='yellow', label='5 < ∆t ≤ 30 min', s=50)
    ax.scatter([], [], c='red', label='∆t > 30 min', s=50)
    ax.scatter([], [], c='gray', label='First Point', s=50)
    ax.legend(loc='lower right', fontsize=10)

    if idle_periods:
        ax.text(-0.12, 0.5, "IDLE:\n" + "\n".join(idle_periods[-5:]),
                transform=ax.transAxes, fontsize=9, color='red',
                bbox=dict(facecolor='white', alpha=0.75))
    if moderate_periods:
        ax.text(1.05, 0.5, "MODERATE:\n" + "\n".join(moderate_periods[-5:]),
                transform=ax.transAxes, fontsize=9, color='goldenrod',
                bbox=dict(facecolor='white', alpha=0.75))

    ax.set_axis_off()
    plt.tight_layout()
    plt.savefig(f"{FRAME_DIR}/frame_{i:04d}.png", dpi=100)
    plt.close()

# === FINAL SUMMARY FRAME ===
fig, ax = plt.subplots(figsize=(10, 10))
ax.axis("off")
ax.text(0.5, 0.92, "Summary: Time Spent Per Block (Including Anomalies)", fontsize=14, ha='center', weight='bold')

text = "\n".join([
    f"{block}: {row['Total_Including_Anomalies']} min" for block, row in summary.iterrows()
])
ax.text(0.05, 0.75, text, fontsize=12, va='top', family='monospace')

if not anomalies_df.empty:
    ax.text(0.55, 0.75, "Anomalies (>5 min):", fontsize=11, weight='bold', color='red')
    for i, row in anomalies_df.iterrows():
        ax.text(0.55, 0.72 - i*0.035,
                f"{row['Start_Time'].strftime('%H:%M')}–{row['End_Time'].strftime('%H:%M')} | {row['Block_Start']} → {row['Block_End']}",
                fontsize=10, color='red')

plt.tight_layout()
plt.savefig(f"{FRAME_DIR}/frame_{len(gdf_points):04d}.png", dpi=100)
plt.close()

# === GENERATE VIDEO ===
os.system(
    f"ffmpeg -framerate {FPS} -i {FRAME_DIR}/frame_%04d.png "
    f"-vf scale=800:-1 -c:v libx264 -pix_fmt yuv420p {VIDEO_NAME}"
)
print(f"\u2705 Video created: {VIDEO_NAME}")


Generating frames: 100%|████████████████████████| 70/70 [01:12<00:00,  1.04s/it]
ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libo

✅ Video created: outputs/gps_animation.mp4


[out#0/mp4 @ 0x7fc206a04b80] video:305KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.549906%
frame=   71 fps= 37 q=-1.0 Lsize=     307KiB time=00:00:34.50 bitrate=  72.9kbits/s speed=18.1x    
[libx264 @ 0x7fc206a05dc0] frame I:1     Avg QP: 9.47  size:273487
[libx264 @ 0x7fc206a05dc0] frame P:19    Avg QP:13.41  size:  1366
[libx264 @ 0x7fc206a05dc0] frame B:51    Avg QP:23.59  size:   248
[libx264 @ 0x7fc206a05dc0] consecutive B-frames:  2.8%  2.8%  4.2% 90.1%
[libx264 @ 0x7fc206a05dc0] mb I  I16..4: 23.0% 21.8% 55.2%
[libx264 @ 0x7fc206a05dc0] mb P  I16..4:  2.9%  0.0%  0.4%  P16..4:  0.7%  0.1%  0.1%  0.0%  0.0%    skip:95.8%
[libx264 @ 0x7fc206a05dc0] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  0.3%  0.1%  0.1%  direct: 0.1%  skip:99.5%  L0:45.9% L1:48.5% BI: 5.6%
[libx264 @ 0x7fc206a05dc0] 8x8 transform intra:13.7% inter:9.6%
[libx264 @ 0x7fc206a05dc0] coded y,uvDC,uvAC intra: 46.6% 44.9% 44.3% inter: 0.1% 0.1% 0.1%
[libx264 @ 0x7fc206a05dc0]